In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import sys
from pathlib import Path
import anndata as ad
import scanpy as sc
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt
sys.path.insert(0, "/home/ylu/project/spateo-release_new")
import spateo_dev as st
sys.path.insert(0, "/home/ylu/project")
from utils import *
import time

os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'

%load_ext autoreload
%autoreload 2

2024-07-11 23:27:19.296388: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-11 23:27:19.477180: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.3/lib64
2024-07-11 23:27:19.477225: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-07-11 23:27:20.198071: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot ope

In [2]:
data_folder = "./results/split_slices/"
results_folder = "./results/Spateo/"
figures_folder = "./results/figures/Spateo"
Path(results_folder).mkdir(parents=True, exist_ok=True)

In [3]:
## load the data
from tqdm import tqdm
slices = []
for i in tqdm(range(59)):
    slices.append(ad.read_h5ad(os.path.join(data_folder, f"slice_{i}.h5ad")))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 59/59 [02:52<00:00,  2.93s/it]


In [4]:
spatial_key = "spatial"
anno_key = "class"
palette = slices[0].uns[f"{anno_key}_palette"]

In [5]:
## Rotate the data
rotate_key = "spatial_rot"
rotations = np.load("./results/random_rotations.npy", allow_pickle=True)
for i in range(len(slices)):
    slices[i].obsm[rotate_key] = slices[i].obsm[spatial_key][:,:2].copy()
    mean = np.mean(slices[i].obsm[rotate_key], axis=0)
    slices[i].obsm[rotate_key] = slices[i].obsm[rotate_key] - mean
    slices[i].obsm[rotate_key] = slices[i].obsm[rotate_key] @ rotations[i].T + mean
    

In [6]:
spatial_key = "spatial_rot"
key_added = "aligned_spatial"

In [8]:
## Perform the Spateo alignment
sampling_num = 20000
for i in tqdm(range(len(slices)-1)):
    slice1, slice2 = slices[i].copy(), slices[i+1].copy()
    sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
    sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
    slice1 = slice1[sampline_idx1,:]
    slice2 = slice2[sampline_idx2,:]
    time_start = time.time()
    align_slices, pis, _  = st.align.morpho_align(
        models=[slice1, slice2],
        spatial_key=spatial_key,
        key_added=key_added,
        device="0",
        mode="SN-S",
        max_iter=200,
        verbose=True,
        partial_robust_level=20,
        beta=1e-5,
        lambdaVF=1e5,
        iter_key_added=None,
    )
    time_end = time.time()
    matches = [np.arange(pis[0].shape[0]), np.argmax(pis[0], axis=1)]
    R, t=solve_RT_by_correspondence(align_slices[1].obsm[key_added], align_slices[1].obsm[spatial_key])
    alignment_results = {'sampling_idx1': sampline_idx1, 'sampline_idx2': sampline_idx2, 'R': R, 't': t, 'matches': matches, 'time': time_end - time_start}
    np.save(os.path.join(results_folder, f"slice_{i}_{i+1}_sampling_{sampling_num}.npy"), alignment_results, allow_pickle=True)

  0%|                                                                                                                                          | 0/58 [00:00<?, ?it/s]

|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%|-----> Filtered all samples for common genes. There are 550 common genes.
|-----> Coordinates normalization params:
|-----------> Scale: 1.3613672256469727.
|-----> Performing coarse rigid alignment...
|-----> Coarse rigid alignment done.
|-----> [Start morpho alignment] in progress: 100.0000%
|-----> [Start morpho alignment] finished [3.6658s]
|-----> Key Parameters: gamma: 0.9900000095367432; beta2: 0.43164917826652527; sigma2: 0.004966384265571833
|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%
|-----> [Models alignment based on morpho, mode: SN-S.] finished [1.3928s]


  2%|██▏                                                                                                                               | 1/58 [00:14<13:34, 14.29s/it]

|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%|-----> Filtered all samples for common genes. There are 550 common genes.
|-----> Coordinates normalization params:
|-----------> Scale: 1.539260745048523.
|-----> Performing coarse rigid alignment...
|-----> Coarse rigid alignment done.
|-----> [Start morpho alignment] in progress: 100.0000%
|-----> [Start morpho alignment] finished [3.4065s]
|-----> Key Parameters: gamma: 0.9900000095367432; beta2: 0.5462127327919006; sigma2: 0.0025663021951913834
|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%
|-----> [Models alignment based on morpho, mode: SN-S.] finished [1.3881s]


  3%|████▍                                                                                                                             | 2/58 [00:28<13:08, 14.08s/it]

...

 97%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌    | 56/58 [13:02<00:27, 13.84s/it]

|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%|-----> Filtered all samples for common genes. There are 550 common genes.
|-----> Coordinates normalization params:
|-----------> Scale: 1.1519076824188232.
|-----> Performing coarse rigid alignment...
|-----> Coarse rigid alignment done.
|-----> [Start morpho alignment] in progress: 100.0000%
|-----> [Start morpho alignment] finished [3.3820s]
|-----> Key Parameters: gamma: 0.9836888313293457; beta2: 0.4106754958629608; sigma2: 0.0436728373169899
|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%
|-----> [Models alignment based on morpho, mode: SN-S.] finished [1.6764s]


 98%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊  | 57/58 [13:17<00:14, 14.05s/it]

|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%|-----> Filtered all samples for common genes. There are 550 common genes.
|-----> Coordinates normalization params:
|-----------> Scale: 1.0157456398010254.
|-----> Performing coarse rigid alignment...
|-----> Coarse rigid alignment done.
|-----> [Start morpho alignment] in progress: 100.0000%
|-----> [Start morpho alignment] finished [2.7847s]
|-----> Key Parameters: gamma: 0.9359550476074219; beta2: 0.43212494254112244; sigma2: 0.03879053518176079
|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%
|-----> [Models alignment based on morpho, mode: SN-S.] finished [0.9716s]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 58/58 [13:26<00:00, 13.91s/it]
